# Keras Mask R-CNN

In [1]:
import numpy as np
import random
import os
import cv2
import time

## Carregando as labels e cores

#### Carregando labels

In [2]:
# Diretorio base
dir_mask_rcnn = "/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/maskRCNN/mask-rcnn-coco"

# Junta o 'dir_mask_rcnn' com o nome do arquivo que tem as labels
labelsPath = os.path.sep.join([dir_mask_rcnn, 'object_detection_classes_coco.txt'])
print(labelsPath)

# Le o arquivo 'labelsPath' e gera as labels
LABELS = open(labelsPath).read().strip().split('\n')
print(LABELS[:5])

/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/maskRCNN/mask-rcnn-coco/object_detection_classes_coco.txt
['person', 'bicycle', 'car', 'motorcycle', 'airplane']


#### Carregando cores

In [3]:
# Carrega as cores do COCO clas
colorsPath = os.path.sep.join([dir_mask_rcnn, 'colors.txt'])
print(colorsPath)

# Le o arquivo 'colorsPath' e gera as cores
COLORS = open(colorsPath).read().strip().split("\n")
COLORS = [np.array(c.split(",")).astype("int") for c in COLORS]
COLORS = np.array(COLORS, dtype='uint8')
print(COLORS)

/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/maskRCNN/mask-rcnn-coco/colors.txt
[[  0 255   0]
 [  0   0 255]
 [255   0   0]
 [  0 255 255]
 [255 255   0]
 [255   0 255]]


## Carregando o modelo

#### Carregando pesos e configurações do modelo

In [4]:
# Carregando os pesos e configurações
weigthsPath = os.path.sep.join([dir_mask_rcnn,'frozen_inference_graph.pb'])
print(weigthsPath)
configPath = os.path.sep.join([dir_mask_rcnn, 'mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'])
print(configPath)

/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/maskRCNN/mask-rcnn-coco/frozen_inference_graph.pb
/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/maskRCNN/mask-rcnn-coco/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt


#### Carregando MASK R-CNN treinada com o dataset COCO dataset (90 classes)

In [5]:
# Carregando 
net = cv2.dnn.readNetFromTensorflow(weigthsPath, configPath)

## Carregando Imagem

In [7]:
# Criando link para carregar imagem
dir_image = '/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/maskRCNN/images'
imagePath = os.path.sep.join([dir_image, 'example_01.jpg'])

# Carregando imagem
image = cv2.imread(imagePath)
print(image.shape)
print(image.shape[:2])
(H, W) = image.shape[:2]

(600, 800, 3)
(600, 800)


In [10]:
# Criando o blob da imagem
# Caixa delimitadora e segmentação da imagem
blob = cv2.dnn.blobFromImage(image, swapRB = True, crop = False)
net.setInput(blob)
start=time.time()
(boxes, masks) = net.forward(['detection_out_final','detection_masks'])
end = time.time()

# Mostrando informações
print("[INFO] Mask R-CNN took {:.6f} seconds".format(end - start))
print("[INFO] boxes shape: {}".format(boxes.shape))
print("[INFO] masks shape: {}".format(masks.shape))

[INFO] Mask R-CNN took 6.522977 seconds
[INFO] boxes shape: (1, 1, 3, 7)
[INFO] masks shape: (100, 90, 15, 15)


## Filtro e visualização

In [23]:
# Loop no número de objetos detectados
for i in range(0, boxes.shape[2]):
    # Extrair a classeID referente a detecção e sua respectiva probabilidade 
    classID = boxes[0, 0, i, 1]
    confidence = boxes[0, 0, i, 2]
    
    # verifica se a probabilidade é maior que 50%
    if confidence > 0.5:
        # faz a cópia da imagem
        clone = image.copy()
        
        # dimensionamos a caixa delimitadora do objeto e calculamos as dimensões da caixa
        box = boxes[0, 0, 1, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype('int')
        boxW = endX - startX
        boxH = endY - startY
        
        # extrai a segmentação em pixels para o objeto, 
        # redimensione a máscara de forma que ela tenha as mesmas 
        # dimensões da caixa delimitadora e, finalmente, limite para criar uma máscara * binária *
        mask = masks[int(i), int(classID)]
        mask = cv2.resize(mask, (boxW, boxH),
                         interpolation = cv2.INTER_NEAREST)
        mask = (mask > 0.3)
        
        # Extrai região de interesse da imagem
        roi = clone[startY:endY, startX:endX]
        
        # converter a máscara de um booleano para uma máscara inteira
        # para valores: 0 ou 255, depois aplique a máscara
        visMask = (mask * 255).astype("uint8")
        instance = cv2.bitwise_and(roi, roi, mask=visMask)
        
        # show the extracted ROI, the mask, along with the
        # segmented instance
        cv2.imshow("ROI", roi)
        cv2.imshow("Mask", visMask)
        cv2.imshow("Segmented", instance)
        